## <랭체인과 llama로 RAG 기반의 챗봇 서비스 구현>
pip install unstructured : 텍스트 파일 같은 구조화되지 않은 데이터를 다루는 데 사용  
pip install sentence-transformers : 문장을 벡터로 변환하고 이를 통해 텍스트 데이터의 의미적 유사성을 계산하기 위해 사용   
pip install chromadb : 벡터를 저장하고 유사도 검색을 지원     

In [2]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install unstructured

In [4]:
!pip install langchain-community

In [5]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [6]:
!pip install chromadb

In [7]:
!pip install openai

In [8]:
!pip show langchain

Name: langchain
Version: 0.2.14
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /home/conet/anaconda3/envs/lchain/lib/python3.10/site-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community


In [9]:
from langchain.document_loaders import TextLoader
loader = TextLoader('/home/conet/다운로드/AI.txt')
documents = loader.load()

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 문서를 청크로 분할 : 큰 덩어리의 문서를 작은 덩어리로 분할하는 과정 / langchain에서 제공하는 RecursiveCharacterTextSplitter 이용
def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_documents(documents)
    return docs

# docs 변수에 분할 문서를 저장
docs = split_docs(documents)


In [11]:
from langchain.embeddings import SentenceTransformerEmbeddings
# 임베딩 : 'all_MiniLM-L6-v2'모델 사용
embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# Chroam(벡터 db)에 임베딩 처리된 벡터 저장
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings)

/home/conet/anaconda3/envs/lchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/conet/anaconda3/envs/lchain/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/conet/.local/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VI

In [12]:
from langchain_community.llms import Ollama
llm = Ollama(model='llama3')

# Q&A 체인을 사용하여 쿼리에 대한 답변 얻기
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type='stuff', verbose=True)

# 쿼리를 작성하고 유사도 검색을 수행하여 답변을 생성, 따라서 텍스트에 있는 내용을 질의해야 합니다.
query = 'AI란?'
# 쿼리와 가장 가까운 벡터(문서)를 반환 (유사도 계산) -> 입력된 쿼리와 가장 유사한 문서를 db에서 찾아냄
matching_docs = db.similarity_search(query)
# chain.run() 메서드를 통해 유사 문서(matching_docs)를 입력으로 받아 질문(query)에 대한 답변 생성
answer = chain.run(input_documents=matching_docs, question=query)
answer

/home/conet/anaconda3/envs/lchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  warn_deprecated(
Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1
/home/conet/anaconda3/envs/lchain/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was dep



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

AI(인공지능) 정보 요약:

AI(인공지능)는 컴퓨터 프로그램이나 기계가 인간의 지능을 모방할 수 있는 능력을 의미합니다. 이 능력에는 학습, 문제 해결, 언어 이해, 패턴 인식, 그리고 의사 결정 등이 포함됩니다.

AI의 주요 유형:
1. 좁은 AI (Narrow AI): 특정 작업을 수행하도록 설계된 AI로, 예를 들어 언어 번역, 이미지 인식, 체스 게임 등이 있습니다.
2. 일반 AI (General AI): 인간의 지적 능력을 광범위하게 재현하는 것을 목표로 하는 AI로, 이론적으로 인간이 할 수 있는 모든 인지적 작업을 수행할 수 있어야 합니다.
3. 생성적 AI (Generative AI): 기존 데이터를 학습하여 새로운 콘텐츠(텍스트, 이미지, 음악 등)를 생성하는 AI의 한 유형입니다.

AI의 응용:
- 의료: 질병 진단, 환자 결과 예측, 개인 맞춤형 치료 등.
- 일상 생활: 가상 비서(Siri, Alexa), 추천 시스템(넷플릭스, 아마존), 자율주행차 등.

Question: AI란?
Helpful Answer:

> Finished chain.

> Finished chain.


'According to the context, AI (인공지능) means the ability of a computer program or machine to mimic human intelligence. This includes abilities such as learning, problem-solving, language understanding, pattern recognition, and decision-making.\n\nSo, the answer is: 인공지능 (AI) - the ability of a computer program or machine to mimic human intelligence.'